In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.regression import *

In [2]:
Dataset_dir = "C:/Users/0stix/Datasets/bike_sharing_demand/"

df_train = pd.read_csv(Dataset_dir+'train.csv')
df_test = pd.read_csv(Dataset_dir+'test.csv')
df_sub = pd.read_csv(Dataset_dir+'sampleSubmission.csv')
len_train = len(df_train)

df_train.drop(['casual','registered'],axis=1,inplace=True)

df_all = pd.concat([df_train,df_test])

In [3]:
df_v1 = df_all.copy()
df_v1['datetime'] = df_v1['datetime'].apply(pd.to_datetime)
df_v1['year'] = df_v1.datetime.apply(lambda x: x.year)
df_v1['month'] = df_v1.datetime.apply(lambda x: x.month)
df_v1['day'] = df_v1.datetime.apply(lambda x: x.day)
df_v1['hour'] = df_v1.datetime.apply(lambda x: x.hour)
df_v1.drop('datetime',axis=1,inplace=True)

ohot_attrib = ['year','month','day','hour','season','holiday','workingday','weather']
df_v1 = pd.get_dummies(df_v1, columns=ohot_attrib)
# df_v1 = pd.get_dummies(df_v1, columns=ohot_attrib, dtype=float64)
df_v1['count'] = df_v1['count'].apply(np.log1p)
df_v1 = df_v1.astype(np.float64)

df_train_v1 = df_v1[:len_train]
df_test_v1 = df_v1[len_train:]
df_test_v1.drop('count',axis=1, inplace=True)

In [4]:
df_train_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10886 entries, 0 to 10885
Data columns (total 86 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   temp          10886 non-null  float64
 1   atemp         10886 non-null  float64
 2   humidity      10886 non-null  float64
 3   windspeed     10886 non-null  float64
 4   count         10886 non-null  float64
 5   year_2011     10886 non-null  float64
 6   year_2012     10886 non-null  float64
 7   month_1       10886 non-null  float64
 8   month_2       10886 non-null  float64
 9   month_3       10886 non-null  float64
 10  month_4       10886 non-null  float64
 11  month_5       10886 non-null  float64
 12  month_6       10886 non-null  float64
 13  month_7       10886 non-null  float64
 14  month_8       10886 non-null  float64
 15  month_9       10886 non-null  float64
 16  month_10      10886 non-null  float64
 17  month_11      10886 non-null  float64
 18  month_12      10886 non-nu

In [6]:
import time
from sklearn.metrics import log_loss

def pycaret_regression(train, target, test, fold):
    start = time.time()
    print('Setup Your Data....')
    setup(data=train,
          target=target,
          use_gpu=False,
          preprocess=False
         )
#     setup(data=train,
#           preprocess=False,
#           target=target,
#           normalize = True,
#           normalize_method = 'robust',
#           silent= True,
#           use_gpu=False,
#           session_id = 123,
#           )
    
#     add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target='pred_proba')

    
    print('Creating Models....')
    
    model_list = []
    lgbm = create_model(estimator = 'lightgbm', fold = fold)
    xgb = create_model(estimator = 'xgboost', fold = fold)
    cat = create_model(estimator = 'catboost', fold = fold)
    
    model_list.append(lgbm)
    model_list.append(xgb)
    model_list.append(cat)
    
    print('Blending Models....')
    blended = blend_models(estimator_list= model_list, fold=fold)
    pred_holdout = predict_model(blended)

    print('Finallizing Models....')
    final_model = finalize_model(blended)

    print('Done...!!!')

#     prep_pipe = get_config('prep_pipe')
#     prep_pipe.steps.append(['trained_model', final_model])
#     pred = prep_pipe.predict_model(test)
    pred = final_model.predict(test)
    print(time.time()-start)
    
    return pred

In [7]:
result = pycaret_regression(df_train_v1, 'count', df_test_v1, 5)

,Description,Value
0,session_id,4431
1,Target,count
2,Original Data,"(10886, 86)"
3,Missing Values,False
4,Numeric Features,16
5,Categorical Features,69
6,Transformed Train Set,"(7620, 85)"
7,Transformed Test Set,"(3266, 85)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


Creating Models....


IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:17:18
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,MAE,MSE,RMSE,R2,RMSLE,MAPE


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: year_2011, year_2012, month_1, month_2, month_3, month_4, month_5, month_6, month_7, month_8, month_9, month_10, month_11, month_12, day_1, day_2, day_3, day_4, day_5, day_6, day_7, day_8, day_9, day_10, day_11, day_12, day_13, day_14, day_15, day_16, day_17, day_18, day_19, hour_0, hour_1, hour_2, hour_3, hour_4, hour_5, hour_6, hour_7, hour_8, hour_9, hour_10, hour_11, hour_12, hour_13, hour_14, hour_15, hour_16, hour_17, hour_18, hour_19, hour_20, hour_21, hour_22, hour_23, season_1, season_2, season_3, season_4, holiday_0, holiday_1, workingday_0, workingday_1, weather_1, weather_2, weather_3, weather_4

In [38]:
df_sub['count'] = result
df_sub.head()
df_sub.to_csv('submission.csv', index = 0)